In [7]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [8]:
with open("nlp.txt", 'r') as f:
    nlp = " ".join(f.read().split('\n'))

# 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [9]:
p = '[.|;|:|?|!]\s+([A-Z])'
sentences = []
split_line = re.split(p, nlp)
sentences.append(split_line[0])
for i in range(1, len(split_line), 2):
    sentences.append(split_line[i] + split_line[i+1])

In [10]:
with open('50.txt', 'w') as f:
    for s in sentences:
        f.write(s) 
        f.write("\n") 

# 51. 単語の切り出し
空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [11]:
with open('51.txt', 'w') as f:
    for s in sentences:
        for w in s.split(' '):
            if w: 
                f.write(w)
                f.write('\n')
        f.write('\n')

# 52. ステミング

51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

In [14]:
from stemming.porter2 import stem
with open('51.txt', 'r') as f:
    words = f.readlines()
with open('52.txt', 'w') as f:
    for w in words:
        word = w[:-1]
        line = word + "\t" + stem(word) + "\n"
        f.write(line)

# 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

In [33]:
# java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -file nlp.txt
import xml.etree.ElementTree as ET
tree = ET.parse('nlp.txt.xml')
root = tree.getroot()

In [47]:
with open('53.txt', 'w') as f:
    for child in root.iter('word'):
        f.write(child.text + "\n")

# 54. 品詞タグ付け
Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [46]:
# ['word', 'lemma', 'CharacterOffsetBegin', 'CharacterOffsetEnd', 'POS', 'NER', 'Speaker']
with open('54.txt', 'w') as f:
    for child in root.iter('token'):
        word = child.find('word').text
        lemma = child.find('lemma').text
        pos = child.find('POS').text
        line = word + "\t" + lemma +"\t" + pos
        f.write(line + "\n")

# 55. 固有表現抽出
入力文中の人名をすべて抜き出せ．

In [49]:
with open('55.txt', 'w') as f:
    for child in root.iter('token'):
        if child.find('NER').text == "PERSON":
            word = child.find('word').text
            f.write(word + "\n")

# 56. 共参照解析
Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

In [73]:
mentions =[[m for m in child.iter('mention')] for child in root.iter('coreference')]
for x in mentions:
    print(x[0].attrib)

{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
{'representative': 'true'}
